In [65]:
import configparser
import os
import pickle

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from scipy.spatial.distance import cosine
from eval_tensor import VerbTensorEvaluator

In [2]:
import matplotlib.pyplot as plt
%pylab inline
pylab.rcParams['figure.figsize'] = (20, 10)

Populating the interactive namespace from numpy and matplotlib


In [3]:
config = configparser.ConfigParser()
config.read('config.ini')
tensor_dir = os.path.join(config['DEFAULT']['ProjectDirectory'], 'tensor/')

In [4]:
evalor = VerbTensorEvaluator(cutoff=50000, decomp_algo='parafac')

In [5]:
evalor.load_embeddings()

In [6]:
subj = pd.Series(evalor.index['nsubj'])

In [7]:
obj = pd.Series(evalor.index['dobj'])

In [88]:
cutoff=80000

In [89]:
vocab = list(subj.head(cutoff).index & obj.head(cutoff).index)

/home/makrai/tool/python/vtensor/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  """Entry point for launching an IPython kernel.


In [90]:
subj_embed = evalor.decomped_tns.factors[0][np.array([subj.loc[w] for w in vocab])]

In [91]:
obj_embed = evalor.decomped_tns.factors[2][np.array([obj.loc[w] for w in vocab])]

In [92]:
#regression = LinearRegression(n_jobs=-1)

In [93]:
#regression.fit(subj_embed, obj_embed)

In [94]:
sim = np.sum(subj_embed*(obj_embed), axis=1)

In [95]:
#diff_norm = np.apply_along_axis(np.linalg.norm, 1, diff.todense())

In [96]:
', '.join([vocab[i] for i in np.argsort(sim.todense())[:10]])

'$, device, apparatus, assembly, portion, plurality, role, module, layer, means'

In [97]:
', '.join([vocab[i] for i in np.argsort(-sim.todense())[:10]])

', he, she, they, that, I, you, we, it, which'

In [103]:
noun = 'role'

In [99]:
df = pd.read_pickle(os.path.join(config['DEFAULT']['ProjectDirectory'], 'dataframe/assoc.pkl')).reset_index()

In [105]:
df[(df.nsubj==noun)].head(10)

,nsubj,ROOT,dobj,freq,freq_nsubj,freq_ROOT,freq_dobj,"freq_('nsubj', 'ROOT')","freq_('nsubj', 'dobj')","freq_('ROOT', 'dobj')",...,log_freq,log_prob,pmi,iact,npmi,niact,pmi_sali,iact_sali,ldice,ldice_sali
1173,role,be,,775928,1689024,1112908392,6348085398,836733,1547738,1058421184,...,19.565563,-14.001164,3.395897,0.913448,0.242544,0.065241,66.442645,17.872123,19.668962,384.834327
12691,role,play,,97133,1689024,60432552,6348085398,102525,1547738,21059850,...,16.567674,-16.999053,4.600873,0.000000,0.270655,0.000000,76.225767,0.000000,16.890396,279.834580
21561,role,advertise,,59385,1689024,1216496,6348085398,59385,1547738,730617,...,15.857811,-17.708916,9.525533,0.141480,0.537895,0.007989,151.054096,2.243563,16.193923,256.800165
21569,role,be,what,59369,1689024,1112908392,144457705,836733,68822,16636995,...,15.857422,-17.709305,5.145359,0.000000,0.290545,0.000000,81.592127,0.000000,18.528174,293.809081
32214,role,change,,40812,1689024,42379328,6348085398,41209,1547738,14837834,...,15.316706,-18.250022,3.861870,0.000000,0.211609,0.000000,59.151130,0.000000,15.643497,239.606843
41012,role,come,,32686,1689024,157022797,6348085398,32686,1547738,152134775,...,14.996385,-18.570342,1.652008,0.831400,0.088959,0.044770,24.774152,12.467999,15.297531,229.407669
48809,role,go,,27650,1689024,240050585,6348085398,27718,1547738,221917718,...,14.754992,-18.811735,0.798250,0.767254,0.042434,0.040786,11.778179,11.320833,15.037845,221.883283
58446,role,include,,23186,1689024,66374884,6348085398,27946,1547738,21910842,...,14.500966,-19.065761,2.398854,0.000000,0.125820,0.000000,34.785702,0.000000,14.822352,214.938428
59456,role,have,,22793,1689024,759840025,6348085398,41660,1547738,226514385,...,14.476303,-19.090424,0.000000,0.001003,0.000000,0.000053,0.000000,0.014525,14.649625,212.072418
87891,role,involve,,15483,1689024,9522387,6348085398,16133,1547738,2807612,...,13.918397,-19.648330,4.617527,0.000000,0.235009,0.000000,64.268582,0.000000,14.252624,198.373680


In [104]:
df[(df.nsubj==noun)|(df.dobj==noun)].head(10)

,nsubj,ROOT,dobj,freq,freq_nsubj,freq_ROOT,freq_dobj,"freq_('nsubj', 'ROOT')","freq_('nsubj', 'dobj')","freq_('ROOT', 'dobj')",...,log_freq,log_prob,pmi,iact,npmi,niact,pmi_sali,iact_sali,ldice,ldice_sali
510,,play,role,1570884,3808605976,60432552,9476829,22787163,4649421,4545100,...,20.583145,-12.983582,6.865188,0.838389,0.528759,0.064573,141.307170,17.256681,21.630734,445.228543
1173,role,be,,775928,1689024,1112908392,6348085398,836733,1547738,1058421184,...,19.565563,-14.001164,3.395897,0.913448,0.242544,0.065241,66.442645,17.872123,19.668962,384.834327
2975,,take,role,349028,3808605976,212256047,9476829,97428701,4649421,651165,...,18.412983,-15.153744,2.882619,0.489052,0.190225,0.032273,53.077608,9.004898,19.405176,357.307173
5355,,have,role,207659,3808605976,759840025,9476829,144079764,4649421,755816,...,17.663857,-15.902870,0.293602,0.177749,0.018462,0.011177,5.186146,3.139728,18.472257,326.291307
6226,he,play,role,180682,484356137,60432552,9476829,5080397,495369,4545100,...,17.463093,-16.103634,6.720259,1.537880,0.417313,0.095499,117.356516,26.856142,21.317538,372.270147
7151,that,play,role,160644,362373329,60432552,9476829,990658,234377,4545100,...,17.293508,-16.273220,6.969265,0.000000,0.428266,0.000000,120.523039,0.000000,21.506554,371.923754
7620,who,play,role,152213,222494631,60432552,9476829,2398169,275625,4545100,...,17.215732,-16.350995,7.595196,0.978734,0.464510,0.059858,130.756857,16.849626,21.992791,378.621989
8224,it,play,role,142667,549081903,60432552,9476829,1054861,209931,4545100,...,17.122292,-16.444435,6.198505,0.000000,0.376936,0.000000,106.132616,0.000000,20.817395,356.441512
9430,they,play,role,126138,446678560,60432552,9476829,3213960,232702,4545100,...,16.944643,-16.622084,6.318641,0.422552,0.380135,0.025421,107.067115,7.159990,20.900651,354.154079
11326,,fill,role,107176,3808605976,9842688,9476829,6693804,4649421,153075,...,16.709622,-16.857105,5.609867,0.670789,0.332789,0.039793,93.738762,11.208628,17.776153,297.032805


In [ ]:
df.loc[:,:,'excellence'].head()

In [ ]:
asymm = pd.Series(np.apply_along_axis(np.linalg.norm, 1, subj_embed - obj_embed),
                 index=vocab)

In [ ]:
#asymm.drop(np.nan, inplace=True)

In [ ]:
asymm.plot()#linewidth=1)
plt.yscale('log')

In [ ]:
asymm.head(1000).plot()#linewidth=1)
plt.yscale('log')

In [ ]:
asymm.head(200).plot()#linewidth=1)
plt.yscale('log')

In [ ]:
np.where(asymm>.5)

In [ ]:
asymm[132:142]

In [ ]:
index['nsubj']['year']

In [ ]:
index['nsubj']['place']

In [ ]:
index['nsubj']['staff']

In [ ]:
index['dobj']['year']

In [ ]:
index['dobj']['place']

In [ ]:
index['dobj']['staff']

In [ ]:
asymm[138]

In [ ]:
asymm.index[138]